In [2]:
COLAB_FLAG = False   # whether running on colab or locally on computer

In [3]:
# if COLAB_FLAG:
#     %pip install pydicom==2.1.2
#     %pip install monai seaborn sentence_transformers
#     %git clone 'https://github.com/mille055/AIPI540_individual_project.git'

    

In [17]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
import torchvision
import pydicom
import pickle
import glob
import sklearn
from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, recall_score, ConfusionMatrixDisplay
from sklearn.linear_model import LogisticRegression

import seaborn as sns
from datetime import datetime

from sentence_transformers import SentenceTransformer

from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn

import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
import time
import os
import copy
from pprint import pprint
from fastai.basics import delegates
from fastcore.parallel import parallel
from fastcore.utils import gt
from fastcore.foundation import L

from pydicom.dataset import Dataset as DcmDataset
from pydicom.tag import BaseTag as DcmTag
from pydicom.multival import MultiValue as DcmMultiValue
import sys
import importlib
import warnings
warnings.filterwarnings("ignore")

In [ ]:
#local imports
if COLAB_FLAG:
    sys.path.append('/content/AIPI540_individual_project/scripts/')
    train_datafile = '/content/AIPI540_individual_project/data/trainfiles.csv'
    val_datafile = '/content/AIPI540_individual_project/data/valfiles.csv'
    test_datafile = '/content/AIPI540_individual_project/data/testfiles.csv'

else: # running locally
    #sys.path.append('/Users/cmm/Documents/GitHub/new/AIPI540_individual_project/scripts/')
    sys.path.append('/workspaces/AIPI540_individual_project/app/scripts/')
    train_datafile = '../data/trainfiles.csv'
    val_datafile = '../data/valfiles.csv'
    test_datafile = '../data/testfiles.csv'


### local imports ###
from config import file_dict, feats, feats_to_keep, column_lists, RF_parameters, classes, model_paths
from config import abd_label_dict, val_list, train_val_split_percent, random_seed, data_transforms
from config import sentence_encoder, series_description_column
from utils import *

from NLP.NLP_training import train_NLP_model
from NLP.NLP_inference import get_NLP_inference
from cnn.cnn_dataset import ImgDataset
from cnn.cnn_inference import image_to_tensor, pixel_inference, test_pix_model, load_pixel_model, visualize_results
from cnn.cnn_model import *
from cnn.cnn_training import *
from cnn.cnn_data_loaders import get_data_loaders
from metadata.meta_inference import meta_inference, calc_feature_importances
from metadata.meta_training import train_fit_parameter_trial, train_meta_model, evaluate_meta_model
from fusionmodel.fus_model import FusionModel
from fusionmodel.fus_inference import get_fusion_inference, get_fusion_inference_from_file
from fusionmodel.fus_training import train_fusion_model, CustomDataset
from model_container import ModelContainer
from process_tree import Processor


: 

: 

In [ ]:


train_df, val_df, test_df = create_datasets(train_datafile, val_datafile, test_datafile)



In [ ]:
# prepares the features (preprocess normalizes/binarizes/ordinalizes the columns and limits to the feature set) and gets the labels
X_train_meta, scaler = preprocess(train_df, need_fit_scaler=True, save_scaler=True)

X_val_meta, _ = preprocess(val_df, scaler)
X_test_meta, _ = preprocess(test_df, scaler)
y_train = train_df.label
y_val = val_df.label
y_test = test_df.label